In [28]:
import requests
res=requests.get("http://data.taipei/opendata/datalist/datasetMeta/download?id=cfa4778c-62c1-497b-b704-756231de348b&rid=87ec7221-bece-49c0-a640-59ca38ad7ad5")

In [29]:
import os 
path="D:\outputXLS"
# path="~/data"
MrtZipName="Mrt.zip"
with open(os.path.join(path,MrtZipName),"wb") as f:
    f.write(res.content)

In [30]:
# import zipfile
# zip_ref = zipfile.ZipFile(r"D:\outputXLS\aaa.zip", 'r')
# zip_ref.extractall(r"D:\outputXLS\aaa")
# zip_ref.close()

import shutil
import zipfile

with zipfile.ZipFile(os.path.join(path,MrtZipName), 'r') as zf:
    for fn in zf.namelist():
        right_fn = fn.encode('cp437').decode('big5')  # 将文件名正确编码
        with open(os.path.join(path,right_fn), 'wb') as output_file:  # 创建并打开新文件
            with zf.open(fn, 'r') as origin_file:  # 打开原文件
                shutil.copyfileobj(origin_file, output_file)  # 将原文件内容复制到新文件

In [31]:
import json
with open(os.path.join(path,right_fn)) as f:
    x=json.load(f)
    
datas=[i['properties'] for i in x['features']]
for j in datas:
    j['name']=j.pop('出入口名稱')
    j['lng']=j.pop('經度')
    j['lat']=j.pop("緯度")
    
ids=[i['項次'] for i in datas]



In [32]:
from pymongo import MongoClient,UpdateOne
operations=[UpdateOne({"_id":idn},{"$set":data},upsert=True) for idn,data in zip(ids,datas)]
client=MongoClient("localhost",27017,username="j122085",password="850605")
db=client.rawData
colle=db.MRTinfo
colle.bulk_write(operations)

[{'lat': 24.959306,
  'lng': 121.418218,
  'name': '頂埔站出口1',
  '出入口編號': '1',
  '項次': 1},
 {'lat': 24.95931, 'lng': 121.419, 'name': '頂埔站出口2', '出入口編號': '2', '項次': 2},
 {'lat': 24.95962, 'lng': 121.4196, 'name': '頂埔站出口3', '出入口編號': '3', '項次': 3},
 {'lat': 24.96039, 'lng': 121.4201, 'name': '頂埔站出口4', '出入口編號': '4', '項次': 4},
 {'lat': 25.063717,
  'lng': 121.552335,
  'name': '松山機場站出口1',
  '出入口編號': '1',
  '項次': 5},
 {'lat': 25.063673,
  'lng': 121.551294,
  'name': '松山機場站出口2',
  '出入口編號': '2',
  '項次': 6},
 {'lat': 25.062923,
  'lng': 121.552241,
  'name': '松山機場站出口3',
  '出入口編號': '3',
  '項次': 7},
 {'lat': 25.060889,
  'lng': 121.544031,
  'name': '中山國中站出口',
  '出入口編號': '0',
  '項次': 8},
 {'lat': 25.041797,
  'lng': 121.543143,
  'name': '忠孝復興站出口1',
  '出入口編號': '1',
  '項次': 9},
 {'lat': 25.041355,
  'lng': 121.543422,
  'name': '忠孝復興站出口2',
  '出入口編號': '2',
  '項次': 10},
 {'lat': 25.041467,
  'lng': 121.544806,
  'name': '忠孝復興站出口3',
  '出入口編號': '3',
  '項次': 11},
 {'lat': 25.041772,
  'lng': 121.54506,


In [266]:
headerStr='''Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8
Accept-Encoding: gzip, deflate
Accept-Language: zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7
Connection: keep-alive
Cookie: JSESSIONID=6353F0F163542ADD743A7800A2180960
Host: data.taipei
Referer: http://data.taipei/opendata/datalist/datasetMeta?oid=1d71c478-205f-42c5-8386-35f86d74fdd1
Upgrade-Insecure-Requests: 1
User-Agent: Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36'''

headers={i.split(":")[0]:i.split(":")[1].strip() for i in headerStr.split("\n")}
res=requests.get("http://data.taipei/opendata/datalist/datasetMeta/download?id=1d71c478-205f-42c5-8386-35f86d74fdd1&rid=0f7126ea-3598-468c-a1aa-bf43b2f483a8",headers=headers)

In [267]:
path="D:\outputXLS"
# path="~/data"
MrtZipName="MrtPeople.csv"
with open(os.path.join(path,MrtZipName),"wb") as f:
    f.write(res.content)

In [268]:
with open(os.path.join(path,MrtZipName), mode='r') as infile:
    peopleMRT=infile.readlines()
x=[]
for i in peopleMRT[1:]:
    y={}
    for title,content in zip(peopleMRT[0].replace("\n","").split(","),i.replace("\n","").split(",")):
        y[title]=content.strip()
        x.append(y)

In [294]:
import pandas
with open(os.path.join(path,MrtZipName), mode='r') as infile:
#     pd=pandas.read_csv(infile)
    df=pandas.read_csv(infile, dtype=int, header=0, parse_dates=[0], thousands=r',')

In [295]:
df['weekday'] = df['日期'].dt.dayofweek

In [310]:
df=df[df['weekday']<5]

In [311]:
df[1:]

,日期,時段,松山機場,中山國中,南京復興,忠孝復興,大安,科技大樓,六張犁,麟光,...,行天宮,東門,蘆洲,三民高中,徐匯中學,三和國中,三重國小,迴龍,丹鳳,weekday
1,2016-12-01,6,216,151,527,431,587,117,113,25,...,475,569,73,117,118,168,68,111,86,3
2,2016-12-01,7,415,891,3180,1620,4729,1032,691,87,...,2459,2239,222,597,391,410,211,512,237,3
3,2016-12-01,8,745,2980,10355,4168,4214,2284,1461,144,...,6586,2732,241,252,329,310,260,360,272,3
4,2016-12-01,9,393,1361,4310,3491,2154,1343,875,112,...,2794,1437,149,173,209,192,174,302,162,3
5,2016-12-01,10,272,571,1834,2759,1070,757,359,93,...,1390,1324,170,175,169,176,212,195,89,3
6,2016-12-01,11,283,457,1596,2860,867,522,318,119,...,1192,1273,177,171,201,198,212,170,151,3
7,2016-12-01,12,286,559,1551,3098,935,595,358,101,...,1246,1480,240,286,312,235,302,191,138,3
8,2016-12-01,13,312,619,1636,3092,941,691,425,123,...,1204,1239,271,256,346,302,329,168,188,3
9,2016-12-01,14,285,538,1372,2706,904,580,378,118,...,1136,1103,311,322,358,325,382,204,227,3
10,2016-12-01,15,253,569,1301,2594,896,519,604,136,...,1014,1060,372,366,404,403,394,270,252,3


In [297]:
df.head()

,日期,時段,松山機場,中山國中,南京復興,忠孝復興,大安,科技大樓,六張犁,麟光,...,行天宮,東門,蘆洲,三民高中,徐匯中學,三和國中,三重國小,迴龍,丹鳳,weekday
0,2016-12-01,5,0,0,0,0,0,0,0,0,...,2,0,0,6,0,0,0,0,0,3
1,2016-12-01,6,216,151,527,431,587,117,113,25,...,475,569,73,117,118,168,68,111,86,3
2,2016-12-01,7,415,891,3180,1620,4729,1032,691,87,...,2459,2239,222,597,391,410,211,512,237,3
3,2016-12-01,8,745,2980,10355,4168,4214,2284,1461,144,...,6586,2732,241,252,329,310,260,360,272,3
4,2016-12-01,9,393,1361,4310,3491,2154,1343,875,112,...,2794,1437,149,173,209,192,174,302,162,3


In [298]:
summaryData=df.fillna(0).groupby("時段").mean()
# summaryData=pd.fillna(0).groupby(["時段",'weekday']).mean()

In [299]:
summaryData

,松山機場,中山國中,南京復興,忠孝復興,大安,科技大樓,六張犁,麟光,辛亥,萬芳醫院,...,行天宮,東門,蘆洲,三民高中,徐匯中學,三和國中,三重國小,迴龍,丹鳳,weekday
時段,,,,,,,,,,,,,,,,,,,,,
0,11.363636,99.772727,123.590909,151.590909,115.636364,95.181818,97.545455,52.090909,33.545455,151.227273,...,126.636364,105.272727,139.636364,189.363636,173.181818,190.318182,158.545455,158.272727,165.409091,2.136364
1,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.136364,9.727273,2.136364
5,0.000000,0.000000,0.318182,0.090909,0.090909,0.000000,0.090909,0.000000,0.000000,0.000000,...,0.136364,0.045455,0.000000,0.318182,0.045455,0.136364,0.045455,0.000000,0.136364,2.136364
6,194.863636,150.272727,480.227273,414.090909,545.545455,117.454545,125.909091,25.863636,28.500000,214.727273,...,468.909091,528.272727,72.454545,107.727273,112.454545,167.727273,70.045455,97.454545,86.227273,2.136364
7,403.818182,844.454545,2980.409091,1554.590909,4671.363636,986.318182,659.318182,72.136364,87.727273,1337.090909,...,2439.500000,2112.045455,229.090909,589.318182,388.272727,409.500000,197.000000,476.409091,218.590909,2.136364
8,737.090909,2878.500000,9895.772727,3994.727273,4049.727273,2253.727273,1390.636364,155.090909,111.863636,1165.500000,...,6280.500000,2631.545455,250.681818,271.045455,305.090909,315.000000,271.909091,377.090909,257.272727,2.136364
9,410.272727,1365.409091,4096.318182,3278.000000,2115.954545,1338.181818,763.590909,130.272727,66.818182,836.000000,...,2748.500000,1466.772727,184.863636,176.500000,210.909091,193.318182,202.136364,306.181818,137.545455,2.136364
10,309.590909,585.409091,1873.136364,3129.454545,1068.363636,776.000000,392.454545,102.818182,49.500000,768.909091,...,1426.727273,1264.818182,183.863636,191.545455,225.818182,173.909091,215.136364,211.590909,118.136364,2.136364
11,309.681818,516.045455,1659.500000,3196.181818,907.409091,562.181818,337.136364,105.090909,51.045455,451.863636,...,1318.090909,1461.363636,211.545455,216.500000,228.181818,210.090909,222.727273,175.090909,147.954545,2.136364


In [312]:
dataDict=summaryData.to_dict()
distinguishMRT=[{"name":i,"lunch":int(dataDict[i][11]+dataDict[i][12]+dataDict[i][13]),"dinner":int(dataDict[i][5]+dataDict[i][6]+dataDict[i][7])} for i in dataDict]

In [313]:
distinguishMRT

[{'dinner': 598, 'lunch': 978, 'name': '松山機場'},
 {'dinner': 994, 'lunch': 1756, 'name': '中山國中'},
 {'dinner': 3460, 'lunch': 5124, 'name': '南京復興'},
 {'dinner': 1968, 'lunch': 10103, 'name': '忠孝復興'},
 {'dinner': 5217, 'lunch': 3032, 'name': '大安'},
 {'dinner': 1103, 'lunch': 2041, 'name': '科技大樓'},
 {'dinner': 785, 'lunch': 1159, 'name': '六張犁'},
 {'dinner': 98, 'lunch': 339, 'name': '麟光'},
 {'dinner': 116, 'lunch': 202, 'name': '辛亥'},
 {'dinner': 1551, 'lunch': 1572, 'name': '萬芳醫院'},
 {'dinner': 55, 'lunch': 143, 'name': '萬芳社區'},
 {'dinner': 1074, 'lunch': 403, 'name': '木柵'},
 {'dinner': 296, 'lunch': 2226, 'name': '動物園'},
 {'dinner': 1366, 'lunch': 1315, 'name': '大直'},
 {'dinner': 661, 'lunch': 1666, 'name': '劍南路'},
 {'dinner': 1195, 'lunch': 1667, 'name': '西湖'},
 {'dinner': 2238, 'lunch': 1133, 'name': '港墘'},
 {'dinner': 1238, 'lunch': 404, 'name': '文德'},
 {'dinner': 283, 'lunch': 719, 'name': '內湖'},
 {'dinner': 98, 'lunch': 309, 'name': '大湖公園'},
 {'dinner': 754, 'lunch': 653, 'name': '葫

In [314]:
MRT=list(colle.find({},{"_id":False}))

In [315]:
for i in MRT:
    station=i['name'].split("站")[0].replace("臺","台")
    try:
        i['lunch']=int(dataDict[station][11]+dataDict[station][12]+dataDict[station][13])
        i['dinner']=int(dataDict[station][5]+dataDict[station][6]+dataDict[station][7])
    except:
        station=station+"站"
        i['lunch']=int(dataDict[station][11]+dataDict[station][12]+dataDict[station][13])
        i['dinner']=int(dataDict[station][5]+dataDict[station][6]+dataDict[station][7])

In [316]:
MRT

[{'dinner': 764,
  'lat': 24.959306,
  'lng': 121.418218,
  'lunch': 487,
  'name': '頂埔站出口1',
  '出入口編號': '1',
  '項次': 1},
 {'dinner': 764,
  'lat': 24.95931,
  'lng': 121.419,
  'lunch': 487,
  'name': '頂埔站出口2',
  '出入口編號': '2',
  '項次': 2},
 {'dinner': 764,
  'lat': 24.95962,
  'lng': 121.4196,
  'lunch': 487,
  'name': '頂埔站出口3',
  '出入口編號': '3',
  '項次': 3},
 {'dinner': 764,
  'lat': 24.96039,
  'lng': 121.4201,
  'lunch': 487,
  'name': '頂埔站出口4',
  '出入口編號': '4',
  '項次': 4},
 {'dinner': 598,
  'lat': 25.063717,
  'lng': 121.552335,
  'lunch': 978,
  'name': '松山機場站出口1',
  '出入口編號': '1',
  '項次': 5},
 {'dinner': 598,
  'lat': 25.063673,
  'lng': 121.551294,
  'lunch': 978,
  'name': '松山機場站出口2',
  '出入口編號': '2',
  '項次': 6},
 {'dinner': 598,
  'lat': 25.062923,
  'lng': 121.552241,
  'lunch': 978,
  'name': '松山機場站出口3',
  '出入口編號': '3',
  '項次': 7},
 {'dinner': 994,
  'lat': 25.060889,
  'lng': 121.544031,
  'lunch': 1756,
  'name': '中山國中站出口',
  '出入口編號': '0',
  '項次': 8},
 {'dinner': 1968,
  'lat':

In [328]:
ids=[i['項次'] for i in MRT]
from pymongo import MongoClient,UpdateOne
operations=[UpdateOne({"_id":idn},{"$set":data},upsert=True) for idn,data in zip(ids,MRT)]
client=MongoClient("192.168.1.113",27017,username="j122085",password="850605")
db=client.rawData
colle=db.MRTinfo
colle.bulk_write(operations)
client.close()

In [325]:
import pandas 
x=pandas.DataFrame(MRT)
x.sort_values("lunch")

,dinner,lat,lng,lunch,name,出入口編號,項次
24,55,24.998580,121.568409,143,萬芳社區站出口,0,25
22,116,25.005119,121.557021,202,辛亥站出口,0,23
60,181,24.973192,121.529950,205,小碧潭站出口2,2,61
59,181,24.971684,121.530311,205,小碧潭站出口1,1,60
156,195,25.137665,121.485373,297,復興崗站出口1,1,157
155,195,25.137325,121.485159,297,復興崗站出口2,2,156
43,98,25.083675,121.602210,309,大湖公園站出口2,2,44
42,98,25.083894,121.602377,309,大湖公園站出口1,1,43
21,98,25.018554,121.558606,339,麟光站出口,0,22
158,384,25.130905,121.473657,366,忠義站出口2,2,159


In [320]:
import pandas as pd
df=pd.DataFrame(MRT)
f = pd.ExcelWriter(r'D:\outputXLS\outputMRT.xlsx')
df.to_excel(f,header=True)
f.save()